# Recording videos of emotions and splitting into .jpg to train
Sources:
- https://www.geeksforgeeks.org/saving-a-video-using-opencv/
- https://stackoverflow.com/questions/30509573/writing-an-mp4-video-using-python-opencv/54731615#54731615
- https://www.youtube.com/watch?v=uL-wCzVMPsc
- https://www.geeksforgeeks.org/cropping-faces-from-images-using-opencv-python/

In [4]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

## Recording Emotion

In [5]:
def record_emotion(emotion='anger', name_face='name/', video_path='./recordings/'):
    cap = cv2.VideoCapture(0)
    cap.set(3,640)
    cap.set(4,480)

    try:
        if not os.path.exists(video_path + name_face):
            os.makedirs(video_path + name_face)
    except OSError:
        print ('Error: Creating directory of' + video_path + name_face)

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    recording_path = video_path + name_face + emotion + '.mp4'
    out = cv2.VideoWriter(recording_path, fourcc, 20.0, (640,480))

    while(True):
        ret, frame = cap.read()
        out.write(frame)
        cv2.imshow('frame', frame)
        c = cv2.waitKey(1)
        if c & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

## Splitting Emotion into .jpg

In [6]:
def split_emotion(emotion='anger', name_face='name/', video_path='./recordings/', frame_path='./frames/'):
    # Playing video from file:
    cap = cv2.VideoCapture(video_path + name_face + emotion + '.mp4')
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    try:
        if not os.path.exists(frame_path + name_face + '/' + emotion):
            os.makedirs(frame_path + name_face + '/' + emotion)
    except OSError:
        print ('Error: Creating directory of' + frame_path + name_face + '/' + emotion)

    currentFrame = 0
    while(currentFrame < length):
        # Capture frame-by-frame
        ret, frame = cap.read()

        if currentFrame > 20 and currentFrame < length - 20 and currentFrame % 3 == 0:
            # Saves image of the current frame in jpg file
            name = frame_path + name_face + emotion + '/' + str(currentFrame) + '.jpg'
            # print ('Creating ' + name)
            cv2.imwrite(name, frame)

        # To stop duplicate images
        currentFrame += 1
    
    print('Split until ' + name)

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

## Cropping Faces from Frames

In [7]:
def crop_face(emotion='anger', name_face='name/', frame_path='./frames/', data_path = './data/', train_path='train/'):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt2.xml')

    # Checks to see if path exists, if not, makedir
    try:
        if not os.path.exists(data_path + name_face + train_path + '/' + emotion):
            os.makedirs(data_path + name_face + train_path + '/' + emotion)
    except OSError:
        print ('Error: Creating directory of' + data_path + name_face + train_path + '/' + emotion)
    
    # for f in [f for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]:
    #     save_faces(cascade, f)
    for file in os.listdir(frame_path + name_face + emotion):
        # print(file)

        # Read the input image
        img = cv2.imread(frame_path + name_face + emotion + '/' + file)
        
        # Convert into grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # print(gray.shape)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        # print(faces)    # coordinates of where face is
        x, y, w, h = faces[0]

        cropped_gray = gray[y:y + h, x : x + w]
        # plt.imshow(cropped_gray, cmap = plt.cm.gray)    # cmap is important as it allows plt.imshow to properly show grayscale
        # print(cropped_gray.shape)

        cv2.imwrite(data_path + name_face + train_path + emotion + '/' + file, cropped_gray)

## Resizing Cropped Images

In [8]:
import random
import shutil

In [9]:
def resize_cropped(emotion='anger', name_face='name/', dim=(48, 48), data_path='./data/', train_path='train/', test_path='test/'):
    
    # Checks to see if path exists, if not, makedir
    try:
        if not os.path.exists(data_path + name_face + train_path + emotion):
            os.makedirs(data_path + name_face + train_path + emotion)
    except OSError:
        print ('Error: Creating directory of' + data_path + name_face + train_path + emotion)
    
    try:
        if not os.path.exists(data_path + name_face + test_path + emotion):
            os.makedirs(data_path + name_face + test_path + emotion)
    except OSError:
        print ('Error: Creating directory of' + data_path + name_face + test_path + emotion)

    for file in os.listdir(data_path + name_face + train_path + emotion):
        # print(file)

        # Read the input image
        img = cv2.imread(data_path + name_face + train_path + emotion + '/' + file)
        resized_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        cv2.imwrite(data_path + name_face + train_path + emotion + '/' + file, resized_img)
        # cv2.imwrite(data_path + name_face + test_path + emotion + '/' + file, resized_img)
    
    source = data_path + name_face + train_path + emotion
    dest = data_path + name_face + test_path + emotion
    files = os.listdir(source)
    no_of_files = len(files) // 5

    for file_name in random.sample(files, no_of_files):
        shutil.move(os.path.join(source, file_name), dest)
        shutil.remove(os.path.join(source, file_name))

## main()

In [58]:
name_face = 'junyi/'
video_path = './recordings/'
frame_path = './frames/'
data_path = './data/'
train_path = 'train/'
test_path = 'test/'
emotion = 'anger'
# emotion = 'happy'
# emotion = 'neutral'
# emotion = 'sad'
# emotion = 'surprise'

# Removed to hopefully allow for more accurate results
# emotion = 'disgust'
# emotion = 'fear'

In [24]:
# record_emotion(emotion, name_face, video_path)
# split_emotion(emotion, name_face, video_path, frame_path)
# crop_face(emotion, name_face, frame_path, data_path, train_path)
# resize_cropped(emotion, name_face, (48, 48), data_path, train_path, test_path)

Split until ./frames/junyi/happy/129.jpg


Error: Destination path './data/junyi/test/happy\96.jpg' already exists

In [25]:
# record_emotion(emotion, name_face, video_path)

In [62]:
split_emotion(emotion, name_face, video_path, frame_path)

Split until ./frames/junyi/anger/120.jpg


In [63]:
# crops faces into ./data/name/train/emotions
crop_face(emotion, name_face, frame_path, data_path, train_path)

In [64]:
# resizes cropped images
resize_cropped(emotion, name_face, (48, 48), data_path, train_path, test_path)
# ^ alreeady copies train to test
# !Xcopy .\train\junyi\happy .\test\junyi\happy